In [ ]:
lfolder = '/mnt/e/sat_papers/terceira/landsat/'
ofolder = '/mnt/e/autocls01/landsat_toar/'

metafile = '/mnt/e/sat_papers/terceira/landsat/LC08_L1TP_216033_20200103_20201016_02_T1_MTL.txt'
metajs   = '/mnt/e/sat_papers/terceira/landsat/LC08_L1TP_216033_20200103_20201016_02_T1_MTL.json'

out_prefix = 'ltoar'

In [ ]:
import os

from glass.pys.oss import lst_ff
from glass.cons.sat import get_ibands
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str
from glass.rst.sat.corr import landsat_toar

In [ ]:
lan_bands = get_ibands('landsat')

In [ ]:
bands = lst_ff(lfolder, file_format='.tif')

ffs = [os.path.splitext(f)[1] for f in bands]

_bands = {}
for f in bands:
    ff = os.path.splitext(f)[1]
    
    for b in lan_bands:
        ref = f'{b}{ff}'
        
        if ref in os.path.basename(f):
            _bands[b] = f
            break

In [ ]:
# Create GRASS GIS Session

loc = f'loc_{now_as_str()}'
gb = run_grass(
    ofolder, grassBIN="grass78", location=loc,
    srs=_bands['B1']
)

import grass.script.setup as gsetup

gsetup.init(gb, ofolder, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.alg import grsrstcalc

In [ ]:
bname = 'band_'
for b in _bands:
    _bands[b] = rst_to_grs(_bands[b], f'{bname}{b[1:]}')
print(_bands)

landsat_toar(bname, f'{out_prefix}_', metafile)

for b in _bands:
    # Go For Integer
    oband = grsrstcalc(
        f'int(round({out_prefix}_{b[1:]} * 10000))',
        f'intband_{b.lower()}'
    )
    
    ofile = grs_to_rst(oband, os.path.join(
        ofolder, f'{out_prefix}_{b.lower()}.tif'
    ), is_int=True)

In [ ]:
_bands